# Numpy-like Operations

@[Xiaoyu Chen](mailto:c-xy17@tsinghua.org.cn)
@[Chaoming Wang](https://github.com/chaoming0625)

Neural modeling often involves the computation of thousands or even millions of elements and requires large-scale computation. To support this, BrainPy provides a basic data structure `brainpy.math.array` with abundant operations. In fact, the concept of `brainpy.math.array` is the same as `ndarray` in NumPy, and the operations used in BrainPy closely resemble NumPy operations. 

```{note}
Users who are not familiar with the Python package NumPy may refer to [https://numpy.org/](https://numpy.org/) for more information.
```

In [3]:
import brainpy.math as bm

bm.set_platform('cpu')

To create an array, users can use several numpy-like operations such as `bm.array()`， `bm.ones()`， `bm.zeros()`， `bm.arange()`， and so on.

In [4]:
bm.array([[0, 1, 2], [3, 4, 5]])

Array([[0, 1, 2],
       [3, 4, 5]], dtype=int32)